## Importación de librerías 

En esta sección se importan las principales librerías utilizadas en el preprocesamiento del texto.

- **spaCy**: biblioteca de procesamiento del lenguaje natural (NLP) que permite realizar tareas como tokenización, eliminación de stopwords o lematización.
    - Se incluye además la configuración personalizada del tokenizador de spaCy, que permite ajustar cómo se separan los tokens (por ejemplo, evitar divisiones innecesarias en palabras con guiones, puntos, etc.).
- **pandas**: biblioteca para la manipulación de datos estructurados (tablas) de tipo DataFrame.

In [9]:
# Librería que contiene herramientas de lematización
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex

# Librería para carga de datos
import pandas as pd

## Definición de funciones y configuración de preprocesamiento

Este bloque incluye todo lo necesario para preparar el texto antes de su análisis. Se utiliza el modelo `en_core_web_lg` de spaCy y se personaliza el tokenizador para evitar divisiones en guiones.

### Componentes definidos:

- **Carga y personalización del modelo spaCy**: se modifica el tokenizador para no dividir palabras por guiones.
- **`tokenizar_texto_spacy(texto)`**: convierte una cadena de texto a minúsculas y la transforma en una lista de tokens.
- **`eliminar_stopwords_spacy(tokens)`**: elimina tokens irrelevantes, incluyendo stopwords, palabras no alfabéticas y una lista personalizada de palabras excluidas.
- **`lematizar_texto_spacy(tokens)`**: transforma cada token en su lema o raíz.
- **`aplicar_mapeo(serie_de_listas, mapeo)`**: aplica un diccionario de equivalencias léxicas para unificar términos similares.
- **`palabras_extra`**: lista manual de palabras irrelevantes específicas del contexto del corpus.
- **`mapeo`**: diccionario de normalización construido a partir de un archivo `.csv` externo.


In [10]:
# Cargar el modelo grande de inglés (incluye vectores de palabras)
nlp = spacy.load("en_core_web_lg")

# Eliminar los guiones como separadores internos para evitar fragmentación de tokens como "ai-based" en "ai" "based"
infixes = [x for x in nlp.Defaults.infixes if "-" not in x]
infix_re = compile_infix_regex(infixes)

# Reconfigurar el tokenizador con las nuevas reglas de infijos
nlp.tokenizer = Tokenizer(
    nlp.vocab,
    rules=nlp.Defaults.tokenizer_exceptions,
    prefix_search=nlp.tokenizer.prefix_search,
    suffix_search=nlp.tokenizer.suffix_search,
    infix_finditer=infix_re.finditer,
    token_match=nlp.tokenizer.token_match
)



# Función de tokenización básica con spaCy en la que también se pasa el texto en minúsculas
# La separación en tokens se realiza según las reglas del modelo de lenguaje cargado,
# teniendo en cuenta espacios, puntuación, signos de puntuación interna (como guiones, apóstrofes, etc.)
# y excepciones predefinidas. En este caso, se ha modificado el tokenizer para que no divida por guiones.
def tokenizar_texto_spacy(texto):
    doc = nlp(texto.lower())
    return list(doc)




# Lista personalizada de palabras a ignorar (además de las stopwords de spaCy)
palabras_extra = [
    "co", "digital", "alexa", "anytime", "cortana", "arduino", "bitcoin",
    "deepl", "digitally", "digitise", "dropbox", "duckduckgo", "e", "etc",
    "eurostat", "example", "facebook", "google", "miro", "t", "twitter",
    "tpm", "youtube", "vs", "whatsapp", "wikipedia", "x", "en", "se", "lego",
    "non", "python", "ros", "scratch", "siri", "-", "non-digital", "one-time",
    "bas", "digital-based", "digitise", "digitised", "digitization", "s", "whilst"
]

# Asegurar que "3d" y "3-d" no se consideren stopwords, ya que el eliminador de stopwrods 
# elimina todas con números
nlp.vocab["3d"].is_alpha = True
nlp.vocab["3-d"].is_alpha = True
nlp.vocab["2d"].is_alpha = True
nlp.vocab["2-d"].is_alpha = True

# Función para eliminar stopwords y tokens no alfabéticos.
# Ignora las palabras_extra
def eliminar_stopwords_spacy(tokens):
    return [
        token for token in tokens
        if not token.is_stop
        and token.is_alpha
        and token.text.lower() not in palabras_extra
    ]
    
    
    
    
# Función de lematización del modelos cargado
def lematizar_texto_spacy(tokens):
    return [token.lemma_ for token in tokens]




# Función de mapeo de palabras en función de un diccionario de mapeo para normalización de términos
def aplicar_mapeo(serie_de_listas, mapeo):
    return serie_de_listas.apply(lambda tokens: [mapeo.get(token, token) for token in tokens])



#carga del archivo que contiene los mapeos correspondientes y conversión a tipo DataFrame para ser tratado como tabla
ruta_mapeo = "../utils/mapping.csv"
df_mapeo = pd.read_csv(ruta_mapeo)

# Construcción del diccionario: original -> equivalente
mapeo = dict(zip(df_mapeo["original"], df_mapeo["equivalente"]))

## Estructuración de datos
- **Datos**: 1 fichero csv con la información de digital skills de ESCO.

- **Texto analizado**:
  - Descripción de la competencia.
  - Preferred Label: nombre principal para identificar la copetencia.

- **Agrupación**:  
  - Broader Concept: grupos a los que pertenecen las competencias.

- **Proceso**:

  1. Para cada competencia, uno el texto de:
     - `description` de la competencia.
     - `preferredLabel`.
  2. Se unen los textos que comparten grupos en la columna `broaderConceptPT`


In [11]:
# Carga del archivo con información de la tabla de ESCO y conversión a tipo DataFrame
ruta_esco = "../data/esco/digitalSkillsCollection_en_4_PropiasDescripcionEliminadas.csv"
esco = pd.read_csv(ruta_esco, encoding="latin1")

esco.head()

,preferredLabel,description,broaderConceptPT
0,use global distribution system,Operate a computer reservations system or a gl...,accessing and analysing digital data
1,use computer telephony integration,Utilise technology that allows interaction bet...,accessing and analysing digital data
2,use ICT systems,Select and use ICT systems for a variety of co...,accessing and analysing digital data
3,handle geospatial technologies,Can use Geospatial Technologies which involve ...,accessing and analysing digital data
4,operate signal generator,Use electronic devices or software tone genera...,accessing and analysing digital data


In [12]:
# Paso 1: Crear una columna con el texto combinado por competencia
esco["Texto"] = esco["preferredLabel"].astype(str) + ' ' + esco["description"].astype(str)

# Paso 2: Agrupar por broaderConceptPT y unir los textos combinados de las competencias
def unir_textos(textos):
    return ' '.join(textos)

esco_agrupado = esco.groupby("broaderConceptPT")["Texto"].agg(unir_textos).reset_index()

display(esco_agrupado)

,broaderConceptPT,Texto
0,CAD software,CADD software The computer-aided design and dr...
1,EU law,GDPR The General Data Protection Regulation is...
2,ICT infrastructure,proxy servers The proxy tools which act as an ...
3,ICT project management methodologies,Agile project management The agile project man...
4,ICT safety,protect the environment from the impact of the...
...,...,...
438,wholesale and retail sales,e-procurement The functioning and methods used...
439,work skills,data quality assessment The process of reveali...
440,working with computers,"manage changes in ICT system Plan, realise and..."
441,working with digital devices and applications,operate digital hardware Use equipment such as...


## Obtención de lemas para la revisión de palabras

El análisis de frecuencias es una técnica fundamental en el preprocesamiento de texto que permite observar qué términos aparecen con mayor recurrencia en un corpus. En este caso, se utiliza para revisar el resultado del procesamiento y facilitar la toma de decisiones sobre dos aspectos clave:

- **Revisión de términos para mapeo léxico:** detectar palabras similares que podrían unificarse mediante reglas de mapeo.
- **Identificación de palabras irrelevantes:** encontrar términos frecuentes pero poco informativos, susceptibles de ser añadidos a una lista personalizada de stopwords.

### Proceso general

   - Preprocesamiento completo del texto (incluido stopwords con palabras adcicionales y mapeos consegudiso hasta el momento)
   - Agrupación de todos los tokens del corpus en el que se obtienen las frecuencias de cada palabra (con función `obtener_frecuencias`).
   - Exportación a archivo externo en el que se pueden analizar manualmente las palabras.

### Utilidad práctica

Este análisis permite:
- Detectar errores o inconsistencias en el preprocesamiento.
- Afinar los criterios de mapeo léxico para mejorar la representación semántica.
- Ampliar la lista de stopwords personalizadas con términos que resulten frecuentes pero irrelevantes en el dominio analizado.

In [13]:
# Paso 1: Unir y concatenar los textos de todas las tablas
texto_concatenado = pd.concat([
    esco_agrupado["Texto"]
], ignore_index=True)

# Paso 2: Aplicar el preprocesamiento completo a cada texto
texto_tokenizado = texto_concatenado.apply(tokenizar_texto_spacy)
texto_filtrado = texto_tokenizado.apply(eliminar_stopwords_spacy)
texto_lematizado = texto_filtrado.apply(lematizar_texto_spacy)
texto_mapeado = aplicar_mapeo(texto_lematizado, mapeo)

# Paso 3: Crear un DataFrame con los textos mapeados para explotar los tokens
tokens = pd.DataFrame({"Texto": texto_mapeado})

# Paso 4: Definición de función para obtener frecuencias
def obtener_frecuencias(df):
    # Explota la lista de palabras de la columna "Texto"
    df_explotado = df.explode("Texto")

    # Renombra la columna para mayor claridad
    df_explotado = df_explotado.rename(columns={"Texto": "Palabra"})

    # Agrupa por palabra y cuenta ocurrencias
    tabla_frecuencias = (
        df_explotado
        .groupby("Palabra")
        .size()
        .reset_index(name="Frecuencia")
        .sort_values(by="Frecuencia", ascending=False)
    )

    return tabla_frecuencias

# Paso 5: Calcular frecuencias
frecuencias = obtener_frecuencias(tokens)

# Paso 6: Exportar a CSV para revisión manual
frecuencias.to_csv("../utils/tokens_revisar_esco.csv", index=False)


## Aplicación del preprocesamiento a los textos de las dimensiones

En este bloque se aplica el flujo de preprocesamiento definido anteriormente (tokenización, eliminación de stopwords, lematización y mapeo) a las distintas tablas de texto correspondientes a varias dimensiones del marco de competencias.

se transforma la columna `"Texto"` de la tabla siguiendo el mismo flujo, con el objetivo de normalizar los textos y prepararlos para análisis posteriores (como comparación semántica).


In [14]:
esco_tokens = esco_agrupado["Texto"].apply(tokenizar_texto_spacy)
esco_sinStopwords = esco_tokens.apply(eliminar_stopwords_spacy)
esco_lematizado = esco_sinStopwords.apply(lematizar_texto_spacy)
esco_agrupado["Texto"] = aplicar_mapeo(esco_lematizado, mapeo)

display(esco_agrupado)

,broaderConceptPT,Texto
0,CAD software,"[cadd, software, design, draft, cadd, use, com..."
1,EU law,"[gdpr, general, datum, protect, regulate, euro..."
2,ICT infrastructure,"[proxy, server, proxy, tool, act, intermediati..."
3,ICT project management methodologies,"[agile, project, manage, agile, project, manag..."
4,ICT safety,"[protect, environment, impact, technology, awa..."
...,...,...
438,wholesale and retail sales,"[function, method, manage, electronic, purchas..."
439,work skills,"[datum, quality, assess, process, reveal, datu..."
440,working with computers,"[manage, change, ict, system, plan, realise, m..."
441,working with digital devices and applications,"[operate, hardware, use, equipment, monitor, m..."


## Funciones para análisis de frecuencias por grupo

Este bloque permite calcular, filtrar y revisar frecuencias de palabras dentro de grupos definidos (por área, competencia, etc.).

### Componentes definidos:

- **`obtener_frecuencias_por_grupo(df, columnas_grupo)`**: genera una tabla de frecuencias por palabra dentro de cada grupo definido.
- **`filtrar_por_umbral_por_grupo(tabla, columnas_grupo)`**: filtra palabras frecuentes aplicando un umbral (media + desviación) dentro de cada grupo.
- **`imprimir_estadisticas_por_grupo(nombre, tabla_original, tabla_filtrada, columnas_grupo)`**: imprime estadísticas básicas del filtrado para cada grupo.


In [15]:
# Función para generar la tabla de frecuencias por grupo
def obtener_frecuencias_por_grupo(df, columnas_grupo):
    # Explota la columna "Texto", que contiene listas de palabras, en filas individuales
    df_explotado = df.explode("Texto")

    # Renombra la columna para que se entienda que cada fila representa una palabra
    df_explotado = df_explotado.rename(columns={"Texto": "Palabra"})

    # Agrupa por las columnas indicadas (grupo) y por palabra, y cuenta ocurrencias
    tabla_frecuencias = (
        df_explotado
        .groupby(columnas_grupo + ["Palabra"])
        .size()
        .reset_index(name="Frecuencia")
    )

    return tabla_frecuencias
    
    
# Función para filtrar palabras por grupo según un umbral
def filtrar_por_umbral_por_grupo(tabla_frecuencias, columnas_grupo):
    
    # Función interna que calcula el umbral y filtra un solo grupo
    def filtrar_grupo(grupo):
        media = grupo["Frecuencia"].mean()
        desviacion = grupo["Frecuencia"].std()
        umbral = media + desviacion
        return grupo[grupo["Frecuencia"] > umbral]
    
    # Aplica el filtrado a cada grupo y devuelve el resultado combinado
    return tabla_frecuencias.groupby(columnas_grupo, group_keys=False).apply(filtrar_grupo).reset_index(drop=True)


# Función para imprimir estadísticas descriptivas por grupo
def imprimir_estadisticas_por_grupo(nombre_tabla, tabla_frecuencias, tabla_frecuencias_filtradas, columnas_grupo):

    print(f"=== {nombre_tabla} ===")
    
    # Recorre cada grupo para calcular sus estadísticas y mostrar comparativas
    for nombre_grupo, grupo in tabla_frecuencias.groupby(columnas_grupo):
        total_original = grupo.shape[0]
        media = grupo["Frecuencia"].mean()
        desviacion = grupo["Frecuencia"].std()
        umbral = media + desviacion
        
        # Crea una máscara lógica para identificar el grupo correspondiente en la tabla filtrada
        if isinstance(nombre_grupo, tuple):
            mascara = True
            for col, val in zip(columnas_grupo, nombre_grupo):
                mascara &= (tabla_frecuencias_filtradas[col] == val)
        else:
            mascara = (tabla_frecuencias_filtradas[columnas_grupo] == nombre_grupo)
            
        grupo_filtrado = tabla_frecuencias_filtradas[mascara]
        total_filtrado = grupo_filtrado.shape[0]

        # Imprime resultados por grupo
        nombre = ", ".join(str(val) for val in nombre_grupo) if isinstance(nombre_grupo, tuple) else str(nombre_grupo)
        print(f"--- Grupo: {nombre} ---")
        print(f"Total de palabras (antes del filtrado): {total_original}")
        print(f"Total de palabras (después del filtrado): {total_filtrado}")
        print(f"Media de frecuencia: {media:.2f}")
        print(f"Desviación típica: {desviacion:.2f}")
        print(f"Umbral aplicado (media + desviación): {umbral:.2f}")
        print()


## Cálculo de frecuencias por dimensión y grupo

Este bloque ejecuta el análisis de frecuencias por grupos definidos en distintas dimensiones del marco competencial. Para cada grupo:

1. Se calcula la frecuencia total de palabras (`esco_sinUmbral`).
2. Se filtran las más relevantes usando un umbral estadístico (`esco_filtrado`).
3. Se imprimen estadísticas básicas para revisión.

In [16]:
esco_sinUmbral = obtener_frecuencias_por_grupo(esco_agrupado, ["broaderConceptPT"])
esco_filtrado = filtrar_por_umbral_por_grupo(esco_sinUmbral, ["broaderConceptPT"])
imprimir_estadisticas_por_grupo("Broader Concept", esco_sinUmbral, esco_filtrado, ["broaderConceptPT"])

C:\Users\mario.fernandezr\AppData\Local\Temp\ipykernel_17916\3679962933.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return tabla_frecuencias.groupby(columnas_grupo, group_keys=False).apply(filtrar_grupo).reset_index(drop=True)


=== Broader Concept ===
--- Grupo: CAD software ---
Total de palabras (antes del filtrado): 20
Total de palabras (después del filtrado): 2
Media de frecuencia: 1.50
Desviación típica: 0.83
Umbral aplicado (media + desviación): 2.33

--- Grupo: EU law ---
Total de palabras (antes del filtrado): 13
Total de palabras (después del filtrado): 3
Media de frecuencia: 1.31
Desviación típica: 0.63
Umbral aplicado (media + desviación): 1.94

--- Grupo: ICT infrastructure ---
Total de palabras (antes del filtrado): 39
Total de palabras (después del filtrado): 6
Media de frecuencia: 1.59
Desviación típica: 1.12
Umbral aplicado (media + desviación): 2.71

--- Grupo: ICT project management methodologies ---
Total de palabras (antes del filtrado): 31
Total de palabras (después del filtrado): 5
Media de frecuencia: 4.68
Desviación típica: 3.51
Umbral aplicado (media + desviación): 8.18

--- Grupo: ICT safety ---
Total de palabras (antes del filtrado): 52
Total de palabras (después del filtrado): 6
Med

## Construcción de matrices de frecuencias por grupo

En este bloque se preparan las matrices de frecuencias necesarias para los análisis posteriores. Para ello:

1. **Se crea una columna combinada `Comp`** que concatena `ID_area` e `ID_competencia` para los casos en los que las agrupaciones sean las competencias.
2. **Se generan matrices de frecuencias** mediante `pivot_table()`, en las que:
   - Las filas corresponden a palabras.
   - Las columnas representan los distintos grupos.
   - Los valores son las frecuencias de aparición de cada palabra en cada grupo.
3. **Se exportan las matrices a archivos `.csv`**.

Este paso transforma los datos textuales en estructuras matriciales.

### Ejemplo simplificado

**Antes del pivoteo:**

| Palabra   | broaderConceptPT | Frecuencia |
|-----------|------------------|------------|
| access    | use software     | 3          |
| digital   | hardware         | 2          |
| access    | ICT software     | 5          |

**Después del pivoteo:**

| Palabra   | use software | hardware |
|-----------|--------------|----------|
| access    | 3            | 5        |
| digital   | 2            | 0        |


In [17]:
# Pivotar la tabla para obtener una matriz
# filas = palabras, columnas = grupos, valores = frecuencias
matriz_frecuencias_esco = esco_filtrado.pivot_table(index="Palabra", columns="broaderConceptPT", values="Frecuencia", fill_value=0)
matriz_frecuencias_esco.columns.name = "Palabra"
matriz_frecuencias_esco.index.name = None
matriz_frecuencias_esco.to_csv("../results/esco/frecuencias_esco.csv")